In [ ]:
import os,sys
import torch
import importlib
from utils import options
from utils.util import log

import importlib
import pdb
import numpy as np
import os, sys, time
import torch
import tqdm
import matplotlib.pyplot as plt 

import utils.util as util
from easydict import EasyDict as edict

from utils.util import log
import utils.camera as camera
from pipelines import base
from pipelines import Camera
from pipelines import Point3D
from pipelines import Initialization
from pipelines import Registration
from pipelines import BA
from pipelines import rendering_refine
from pipelines import Initialization_Trad
from pipelines import BA_Trad
from pipelines import Registration_Trad
from notebooks import vis_3d

from IPython.display import HTML


In [ ]:
argv = ['--group=ETH3D', '--pipeline=LevelS2fM', '--yaml=ETH3D', '--name=facade', '--data.dataset=ETH3D', '--data.scene=courtyard', '--sfm_mode=full', '--nbv_mode=ours', '--refine_again=false']
opt_cmd = options.parse_arguments(argv)
opt = options.set(opt_cmd=opt_cmd,safe_check=False)

module = importlib.import_module("pipelines.{}".format(opt.pipeline))
model_ours = module.Model(opt)
model_ours.load_dataset(opt)
model_ours.restore_checkpoint(opt)
model_ours.setup_visualizer(opt)


In [ ]:
var = model_ours.load_matches(opt)
pose_graph_path = f"data/{opt.data.dataset}/{opt.data.scene}/pose_graph.npy"
pose_graph = np.load(pose_graph_path, allow_pickle=True)[:]
var.indx_init = pose_graph[[0,1]]
var.imgs_init = model_ours.train_data.all.image[var.indx_init]
var.kypts_init = [var.kypts[i] for i in var.indx_init]
var.intrs_init = model_ours.train_data.all.intr[var.indx_init]
var.mchs_init = [var.matches[i] for i in var.indx_init]
var.inliers_init = [var.masks[i] for i in var.indx_init]
var.gt_depths = None
var.omn_depths = model_ours.train_data.all.depth_omnidata[var.indx_init]
var.omn_norms = model_ours.train_data.all.norm_omnidata[var.indx_init]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
axes[0].imshow(var.imgs_init[0].permute((1,2,0)).cpu().numpy())
axes[0].axis('off')
axes[0].set_title('The First Image')
axes[0].scatter(var.kypts_init[0][:,0].cpu().numpy(), var.kypts_init[0][:,1].cpu().numpy(),s=0.5)
axes[1].imshow(var.imgs_init[1].permute((1,2,0)).cpu().numpy())
axes[1].scatter(var.kypts_init[1][:,0].cpu().numpy(), var.kypts_init[1][:,1].cpu().numpy(),s=0.5)
axes[1].axis('off')
axes[1].set_title('The Second Image')


In [ ]:
Initializer_ours = Initialization.Initializer(opt,
    model_ours.camera_set,
    model_ours.point_set,
    model_ours.sdf_func,
    model_ours.color_func,
    var,cam_info_reloaded=model_ours.cam_info_reloaded)
Initializer_ours.run(
    model_ours.camera_set,
    model_ours.point_set,
    model_ours.sdf_func,
    model_ours.color_func,
    Renderer=model_ours.Renderer,
)

## Reconstructed Two-View Point Clouds 

In [ ]:
camera0 = model_ours.camera_set.cameras[0]
camera1 = model_ours.camera_set.cameras[1]
pose0 = camera0.get_pose().detach()[0]
pose1 = camera1.get_pose().detach()[0]
points3D_ours = torch.cat([_.xyz for _ in model_ours.point_set.pointset])

fig = vis_3d.init_figure()
vis_3d.plot_points(fig, points3D_ours.cpu().numpy(), color='rgba(255,0,255,1)', ps=1, name='Ours')
vis_3d.plot_camera(fig, pose0.cpu().numpy()[:3,:3], pose0.cpu().numpy()[:3,3], camera0.intrinsic.cpu().numpy(), color='rgba(0,255,0,1)', name='Camera 0')
vis_3d.plot_camera(fig, pose1.cpu().numpy()[:3,:3], pose1.cpu().numpy()[:3,3], camera1.intrinsic.cpu().numpy(), color='rgba(0,128,64,1)', name='Camera 1')

fig.show()

In [ ]:
import pyquaternion

def slerp(pose0, pose1, t):
    quat0 = pyquaternion.Quaternion._from_matrix(matrix=pose0[:3,:3].cpu().numpy(),rtol=1e-5, atol=1e-5)
    quat1 = pyquaternion.Quaternion._from_matrix(matrix=pose1[:3,:3].cpu().numpy(),rtol=1e-5, atol=1e-5)
    quatt = pyquaternion.Quaternion.slerp(quat0, quat1, t)
    R = torch.tensor(quatt.rotation_matrix,dtype=pose0.dtype,device=pose0.device)
    T = (1 - t) * pose0[:3,3] + t * pose1[:3,3]
    return torch.cat([R, T[None,:].T], dim=1)

In [ ]:
rendered_seq_rgb = []
rendered_seq_depth = []
rendered_seq_normal = []
height, width = camera0.opt.data.image_size
for t in np.linspace(0, 1, 30):
    rendered_output = camera0.render_img_by_slices(
            model_ours.sdf_func,
            model_ours.color_func,
            model_ours.Renderer,
            slerp(pose0, pose1, t)[None],
            )
    rendered_rgb = rendered_output['rgb'][0].reshape(height,width,3).cpu().numpy()
    rendered_depth = rendered_output['depth'][0].reshape(height,width).cpu().numpy()
    rendered_normal = rendered_output['norm'][0].reshape(height,width,3).cpu().numpy()

    rendered_seq_rgb.append(rendered_rgb)
    rendered_seq_depth.append(rendered_depth)
    rendered_seq_normal.append(rendered_normal)


In [ ]:
def create_animation(frames):
    from matplotlib.animation import FuncAnimation
    fig, ax = plt.subplots()
    ax.set_axis_off()
    if frames.dtype == 'float32':
        min = np.min(frames)
        max = np.max(frames)
        if min < 0 or max > 1:
            frames = (frames - min) / (max - min)
    
    im = ax.imshow(frames[0], cmap='gray')
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
    
    def update(i):
        im.set_data(frames[i])
        return im,

    anim = FuncAnimation(fig, update, frames=frames.shape[0], interval=30, blit=True)
    return anim


In [ ]:
rgb_video = create_animation(np.stack(rendered_seq_rgb,axis=0))
depth_video = create_animation(np.stack(rendered_seq_depth,axis=0))
normal_video = create_animation(np.stack(rendered_seq_normal,axis=0))

### Rendered Two-View Videos

#### RGB

In [ ]:
HTML(rgb_video.to_jshtml())

#### Depth

In [ ]:
HTML(depth_video.to_jshtml())

#### Normal

In [ ]:
HTML(normal_video.to_jshtml())